https://gist.github.com/MaxHalford/9bfaa8daf8b4bc17a7fb7ba58c880675

In [151]:
 # Binary Classification with Sonar Dataset: Baseline
import numpy as np
import pandas as pd

import os

from keras import models

import joblib

from sklearn.metrics import accuracy_score

In [152]:
result_path=os.path.abspath(r'D:\PythonJupyter\MyDev\FinPythonLab\ML-S50F\ML-S50F\Lab-S5-F-DNN\results')

ml_path=os.path.abspath(r'D:\PythonJupyter\MyDev\FinPythonLab\ML-S50F\ML-S50F\Lab-S5-F-DNN\models')

#pkl_file=r'\long_dnn_17x34x17_tn2007-2018_ts2019_130520_1812_pipeline.pkl'
#h5model_file=r'\long_dnn_17x34x17_tn2007-2018_ts2019_130520_1812_model.h5'

h5model_file=r'\long_MLP-Pipeline_17x34x17_epo150_btch32__tn2007-2018_ts2019_180520_1944_model.h5'
pkl_file=r'\long_MLP-Pipeline_17x34x17_epo150_btch32__tn2007-2018_ts2019_180520_1944_pipeline.pkl'


In [153]:
filename='ML-Long_S50M15_07To1Q20-Train.csv'
filepath=os.path.abspath(f'D:\Dev-Trong\DataSet\/{filename}')
print(filepath)

D:\Dev-Trong\DataSet\ML-Long_S50M15_07To1Q20-Train.csv


In [154]:
dataframe = pd.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
dataframe.drop(['open','high','low','close'],axis=1,inplace=True)
print(dataframe.info())
dataframe.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69679 entries, 2007-01-03 09:45:00 to 2020-03-31 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             69679 non-null  float64
 1   indy_ma-1100            69679 non-null  float64
 2   indy_hh-550             69679 non-null  float64
 3   indy_ll-550             69679 non-null  float64
 4   indy_mid-550            69679 non-null  float64
 5   indy_hh2-1100           69679 non-null  float64
 6   indy_ll2-1100           69679 non-null  float64
 7   indy_mid2-1100          69679 non-null  float64
 8   indy_macd110-440        69679 non-null  float64
 9   indy_signal110-440-110  69679 non-null  float64
 10  indy_hist_macd110-440   69679 non-null  float64
 11  indy_rsi25-ma20         69679 non-null  float64
 12  indy_6ATRTrail_DC-110   69679 non-null  float64
 13  cate_3trend-550_ma110   69679 non-null  int64  
 14  cat

(69679, 18)

In [155]:
df=dataframe['01-2019':'12-2020']
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6824 entries, 2019-01-02 09:45:00 to 2020-03-31 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             6824 non-null   float64
 1   indy_ma-1100            6824 non-null   float64
 2   indy_hh-550             6824 non-null   float64
 3   indy_ll-550             6824 non-null   float64
 4   indy_mid-550            6824 non-null   float64
 5   indy_hh2-1100           6824 non-null   float64
 6   indy_ll2-1100           6824 non-null   float64
 7   indy_mid2-1100          6824 non-null   float64
 8   indy_macd110-440        6824 non-null   float64
 9   indy_signal110-440-110  6824 non-null   float64
 10  indy_hist_macd110-440   6824 non-null   float64
 11  indy_rsi25-ma20         6824 non-null   float64
 12  indy_6ATRTrail_DC-110   6824 non-null   float64
 13  cate_3trend-550_ma110   6824 non-null   int64  
 14  cate

In [156]:
labelName=df.columns[-1]
labelName

'LongSignal'

In [157]:

X_df = df.drop(columns=labelName).astype(float)
Y_df = df[labelName]

In [158]:
X_df.head(3)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2019-01-02 09:45:00,1076.48,1085.09,1123.2,1022.6,1072.9,1125.0,1022.6,1073.8,-29.84,-24.72,-5.12,45.39,1049.31,1.0,0.0,3.0,2.0
2019-01-02 10:00:00,1076.41,1085.02,1123.2,1022.6,1072.9,1124.6,1022.6,1073.6,-29.91,-24.80,-5.11,45.98,1049.19,1.0,0.0,3.0,2.0
2019-01-02 10:15:00,1076.35,1084.94,1123.2,1022.6,1072.9,1124.0,1022.6,1073.3,-29.97,-24.88,-5.09,46.57,1049.23,1.0,0.0,3.0,2.0


In [159]:
Y_df.head(3)

datetime
2019-01-02 09:45:00    0
2019-01-02 10:00:00    0
2019-01-02 10:15:00    0
Name: LongSignal, dtype: int64

In [160]:
# split into input (X) and output (Y) variables
X = X_df.to_numpy()
y =Y_df.to_numpy()
print(X.shape)
print(y.shape)

(6824, 17)
(6824,)


In [161]:

pkl_path=f'{ml_path}{pkl_file}'
print(f'{pkl_path}',os.path.exists(pkl_path))
      
h5model_path=f'{ml_path}{h5model_file}'
print(f'{h5model_path}',os.path.exists(h5model_path))



D:\PythonJupyter\MyDev\FinPythonLab\ML-S50F\ML-S50F\Lab-S5-F-DNN\models\long_MLP-Pipeline_17x34x17_epo150_btch32__tn2007-2018_ts2019_180520_1944_pipeline.pkl True
D:\PythonJupyter\MyDev\FinPythonLab\ML-S50F\ML-S50F\Lab-S5-F-DNN\models\long_MLP-Pipeline_17x34x17_epo150_btch32__tn2007-2018_ts2019_180520_1944_model.h5 True


In [162]:

pipe = joblib.load(pkl_path)
print('Before Adding MLP ',pipe)


Before Adding MLP  Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)


In [163]:

model = models.load_model(h5model_path)
print('MLP ',model)



MLP  <keras.engine.sequential.Sequential object at 0x0000025A56601708>


In [164]:


pipe.steps.append(('mlp', model))
print('After Added MLP ',pipe)

After Added MLP  Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.engine.sequential.Sequential object at 0x0000025A56601708>)],
         verbose=False)


In [165]:
pred = pipe.predict_proba(X)[:, 0]



In [166]:
prediction_df = pd.DataFrame(data={
    'acutal_y': y.astype(int),
    'predictProb_y': pred
}, index=X_df.index)


In [167]:
prediction_df['predict_y']=np.where(prediction_df['predictProb_y']>=0.5,1,0)
prediction_df.sample(10)

,acutal_y,predictProb_y,predict_y
datetime,,,
2019-02-26 12:00:00,1,0.748519,1
2019-01-02 15:45:00,0,0.001591,0
2019-11-29 16:30:00,0,0.008262,0
2019-05-29 14:30:00,0,0.025337,0
2019-12-12 16:30:00,0,0.017651,0
2019-08-06 12:15:00,0,0.000258,0
2019-12-16 09:45:00,0,0.029941,0
2019-12-20 16:30:00,0,0.073687,0
2019-09-03 09:45:00,1,0.240793,0


In [168]:
from sklearn.metrics import accuracy_score

round(accuracy_score(prediction_df.acutal_y, prediction_df.predict_y,normalize=True)*100,2)

86.71

In [169]:
df_accurary=prediction_df.query('acutal_y==predict_y')
round((df_accurary.shape[0]/prediction_df.shape[0])*100,2)

86.71

In [170]:


resultfile=h5model_file.replace('model.h5','resultA2')
resultfile=f'{resultfile}.csv'

result_path=f'{result_path}{resultfile}'
result_path

'D:\\PythonJupyter\\MyDev\\FinPythonLab\\ML-S50F\\ML-S50F\\Lab-S5-F-DNN\\results\\long_MLP-Pipeline_17x34x17_epo150_btch32__tn2007-2018_ts2019_180520_1944_resultA2.csv'

In [171]:
prediction_df.to_csv(result_path)